In [96]:
import openai
import os
import re

In [97]:
os.chdir('C:/Users/kaosh/Desktop/Side-project-ML/sentence_analyze')  # 換路徑使用

In [98]:
import openpyxl
wb = openpyxl.load_workbook('TCB Response.xlsx', data_only=True)     # 開啟 Excel 檔案



In [99]:
s1 = wb['Innovation1']        # Get the worksheet content

In [100]:
# Store all answer and criteria in an array
s1_read = s1.iter_rows(min_col= 2, max_col=3)
Domain = []
for row in s1_read:
    tmp = []
    for column in row:
        tmp.append(column.value)
    Domain.append(tmp)
    
# Store all questions in an array
s1_question_read = s1.iter_rows(max_col=1)
question = []
for row in s1_question_read:
    tmp = []
    for column in row:
        tmp.append(column.value)
    question.append(tmp)


In [101]:
# Set your OpenAI API key
openai.api_key = '...'

In [102]:
# The analyze_sentence function sends a request to the OpenAI API with the given sentence and criteria, and returns the decision made by the model.
def analyze_sentence(sentence, criteria):
    response = openai.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=f"Sentence: {sentence}\nCriteria: {criteria}\nDecision:",
        temperature=0.7,
        max_tokens=100,
        stop=["\n"]
    )
    decision = response.choices[0].text.strip()
    return decision

In [103]:
def summarize_paragraph(paragraph):
    # Use the completion endpoint to summarize the paragraph
    response = openai.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=f"Write a concise summary of the following: {paragraph}",
      max_tokens=100,
      temperature=0.5,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    
    # Extract and return the summarized text from the response
    return response.choices[0].text.strip()

In [104]:
def Average(lst): 
    return sum(lst) / len(lst)

In [105]:
# The main function prompts the user to input the sentence and criteria they want to analyze, calls analyze_sentence with these inputs, and prints out the decision.
run_times = 10 # 期望跑的次數
result = [[] for _ in range(len(Domain)+1)]
result[0].append("Questions")
result[0].append("All Answers concatenate")
result[0].append("Criteria")
for run_time in range(run_times):
    result[0].append(str(run_time + 1))
result[0].append("Average Score")
result[0].append("Final Justification")
for i in range(len(Domain)):
    result[i+1].append(question[i][0]) # Put Questions array in result
    result[i+1].append(Domain[i][0])   # Put all answer array in result
    result[i+1].append(Domain[i][1])   # Put all criteria array in result 
    
    All_justification = "" # Store all justification made by GPT
    for run_time in range(run_times):
        if Domain[i][0] == None:
            result[i+1].append("")
        else: 
            if Domain[i][0].strip("/") == "":
                result[i+1].append("")
            else:
                sentence = Domain[i][0]
                criteria = "What percentage is the sentences meet the criteria: " + Domain[i][1]
                decision = analyze_sentence(sentence, criteria)
                result[i+1].append(f'{decision}')
                All_justification = All_justification + f'{decision}' + "/" # Concatenance all justification into a paragraph 
                print(f'{decision}')
    temp = re.findall(r'\d+%', All_justification)
    res = []
    for n in temp:
        res.append(int(n.strip('%')))
    if len(res) == 0:
        result[i+1].append("")
    else:
        result[i+1].append(Average(res))
    if All_justification == "":
        result[i+1].append("")
    else:
        summary= summarize_paragraph(All_justification)
        result[i+1].append(summary)
        print(summary)

90%
100%
100%
100%
100%



100%
100%
100%
50%
100%

The data shows that for a set of 10 values, 90% of them are 100%, while the remaining 10% are split between 100% and 50%.

80% of the sentences meet the criteria: Clearly Defined Strategy: Communication and departmental understanding of innovation strategy.<br>80% of the sentences meet the criteria: Executive Buy-in: Examples of strategic alignment and execution.
70% of the sentences meet the criteria of clearly defined strategy and executive buy-in. The sentences provide examples of how communication and education through meetings and training are used to convey the cloud vision and strategy to different departments and branches. Additionally, the involvement of senior executives and the digital development council in promoting and executing the strategy further demonstrates their buy-in and alignment with the innovation strategy. However, some sentences do not directly mention the strategy, and the use of consensus camps may not be 

The percentage of sentences that meet the criteria is 100%. Both sentences mention the involvement of industry executives and professional consultants in evaluating the organization's strategies and aligning with investment strategies. Additionally, the second sentence specifically mentions the evaluation process and the need to benchmark it effectively. This shows a clear focus on involving relevant departments and stakeholders in the evaluation process, aligning with investment strategies and ensuring a thorough organizational approach to startup evaluation.
The sentences meet the criteria.
The sentences meet the criteria as they discuss the involvement of industry executives and professional consultants in evaluating the organization's innovative ideas and aligning them with the organization's future strategies and standards. They also mention the need for regular evaluation of implementation methods to ensure alignment with mid- and long-term goals. Additionally, the last sentence 

50%
75%
50%
50% of the sentences meet the criteria. The first sentence references the process of conducting interviews and incorporating user feedback, which aligns with the use of design thinking and customer empathy. However, the second sentence does not directly mention the use of lean hypothesis testing in the project, only referencing user testing and adjustments.
50%

This sentence partially meets the criteria of using design thinking by considering customer empathy in development and conducting user testing and adjustments. However, it does not fully meet the criteria of lean hypothesis application as it does not mention specific examples. Overall, it meets 50% of the criteria.
33.33%
33.33%
The sentences meet the criteria as they mention the Digital Development Committee, which serves as the designated innovation manager and regularly reports on their progress. The mention of a seminar also suggests that there is a focus on utilizing principles and roadmaps for innovation. Howe

In [106]:
excel = openpyxl.Workbook('Innovation_Result.xlsx')     # 開啟 Excel 檔案
s3 = excel.create_sheet() 
for w in result:
    s3.append(w)
excel.save('Innovation_Result.xlsx')

In [107]:
'''
import re
test_string = "33% (1 out of 3 sentences meet the criteria.75% of the sentences meet the criteria. The sentence about cooperating with the Compliance Department may not directly contribute to scripted remediations or a unified system for audits, but it does address the importance of addressing weaknesses and handling reports in a timely and compliant manner. This overall improves compliance posture and can potentially lead to more effective scripted remediations and a more unified system for audits.75% of the sentences meet the criteria for scripted remediations and a unified system for audits. While cooperating with the Compliance Department may not directly contribute to these goals, it does emphasize the importance of addressing weaknesses and handling reports promptly and in compliance, ultimately improving overall compliance posture and potentially leading to more effective remediations and a unified audit system.)"
temp = re.findall(r'\d+%', test_string)
print(temp)
res = []
for i in temp:
    res.append(int(i.strip('%')))
print(Average(res))
'''

'\nimport re\ntest_string = "33% (1 out of 3 sentences meet the criteria.75% of the sentences meet the criteria. The sentence about cooperating with the Compliance Department may not directly contribute to scripted remediations or a unified system for audits, but it does address the importance of addressing weaknesses and handling reports in a timely and compliant manner. This overall improves compliance posture and can potentially lead to more effective scripted remediations and a more unified system for audits.75% of the sentences meet the criteria for scripted remediations and a unified system for audits. While cooperating with the Compliance Department may not directly contribute to these goals, it does emphasize the importance of addressing weaknesses and handling reports promptly and in compliance, ultimately improving overall compliance posture and potentially leading to more effective remediations and a unified audit system.)"\ntemp = re.findall(r\'\\d+%\', test_string)\nprint(